<a href="https://colab.research.google.com/github/Philippe-AD/Jupyter/blob/master/spacy_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy-lefff

In [ ]:
!python -m spacy download fr

In [13]:
import spacy
nlp = spacy.load('en')
nlp = spacy.load('fr')

In [15]:
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr')
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
doc = nlp(u"Apple cherche a acheter une startup anglaise pour 1 milliard de dollard")
for d in doc:
    print(d.text, d.pos_, d._.lefff_lemma, d.tag_, d.lemma_)



2020-09-04 07:01:32,686 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-09-04 07:01:32,687 - spacy_lefff.lefff - INFO - Token lefff_lemma already registered
2020-09-04 07:01:32,687 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-09-04 07:01:33,278 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer
Apple PROPN None PROPN___ Apple
cherche NOUN cherche NOUN__Gender=Fem|Number=Sing cherche
a AUX None AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin avoir
acheter VERB acheter VERB__VerbForm=Inf acheter
une DET un DET__Definite=Ind|Gender=Fem|Number=Sing|PronType=Art un
startup NOUN None NOUN__Gender=Fem|Number=Sing startup
anglaise NOUN anglaise NOUN__Gender=Fem|Number=Sing anglaise
pour ADP None ADP___ pour
1 NUM None NUM__NumType=Card 1
milliard NOUN milliard NOUN__Gender=Masc|Number=Sing|NumType=Card milliard
de ADP un ADP___ de
dollard ADJ None ADJ__Number=Plur dollard




```
# the text to be tokenized 
text = ("Natural language processing (NLP) is a field " +
	"of computer science, artificial intelligence " +
	"and computational linguistics concerned with " +
	"the interactions between computers and human " +
	"(natural) languages, and, in particular, " +
	"concerned with programming computers to " +
	"fruitfully process large natural language " +
	"corpora. Challenges in natural language " +
	"processing frequently involve natural " +
	"language understanding, natural language" +
	"generation frequently from formal, machine" +
	"-readable logical forms), connecting language " +
	"and machine perception, managing human-" +
	"computer dialog systems, or some combination " +
	"thereof.") 
  ```

In [18]:
import spacy # tested with v2.2.3
from spacy.pipeline import EntityRecognizer

text = "Jane lives in Boston. Jan lives in Bremen."

# load the English and German models
nlp_en = spacy.load('en_core_web_sm')  # NER tags PERSON, GPE, ...
nlp_de = spacy.load('fr') # NER tags PER, LOC, ...

# the Vocab objects are not the same
assert nlp_en.vocab != nlp_de.vocab

# but the vectors are identical (because neither model has vectors)
assert nlp_en.vocab.vectors.to_bytes() == nlp_de.vocab.vectors.to_bytes()

# original English output
doc1 = nlp_en(text)
print([(ent.text, ent.label_) for ent in doc1.ents])
# [('Jane', 'PERSON'), ('Boston', 'GPE'), ('Bremen', 'GPE')]

# original German output (the German model makes weird predictions for English text)
doc2 = nlp_de(text)
print([(ent.text, ent.label_) for ent in doc2.ents])
# [('Jane lives', 'PER'), ('Boston', 'LOC'), ('Jan lives', 'PER'), ('Bremen', 'LOC')]

# initialize a new NER component with the vocab from the English pipeline
ner_de = EntityRecognizer(nlp_en.vocab)

# reload the NER component from the German model by serializing
# without the vocab and deserializing using the new NER component
ner_de.from_bytes(nlp_de.get_pipe("ner").to_bytes(exclude=["vocab"]))

# add the German NER component to the end of the English pipeline
nlp_en.add_pipe(ner_de, name="ner_de")

# check that they have the same vocab
assert nlp_en.vocab == ner_de.vocab

# combined output (English NER runs first, German second)
doc3 = nlp_en(text)
print([(ent.text, ent.label_) for ent in doc3.ents])
# [('Jane', 'PERSON'), ('Boston', 'GPE'), ('Jan lives', 'PER'), ('Bremen', 'GPE')]

[('Jane', 'PERSON'), ('Boston', 'GPE'), ('Bremen', 'GPE')]
[('Jane lives in Boston', 'MISC')]
[('Jane', 'PERSON'), ('Boston', 'GPE'), ('Jan', 'PER'), ('Bremen', 'GPE')]
